In [ ]:
#1. 필요한 라이브러리 불러오기

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

#2. 데이터 로드
data = pd.read_csv("../../../../monunmon.csv")  # CSV 파일 경로

#3. 레이블 이진 분류를 위해 변환 (0~94 -> 1, -1 -> 0)
data['Label'] = data['Label'].apply(lambda x: 1 if x >= 0 else 0)

#4. 변환 결과 확인
print("Label Distribution:")
print(data['Label'].value_counts())

#5. 특성과 레이블 분리
X = data.drop(columns=['Label'])  # 피처 데이터
y = data['Label']  # 레이블 데이터

#6. Train, Validation, Test 데이터 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

#7. 데이터 정규화 (CatBoost는 자체적으로 정규화를 처리하므로 이 과정은 생략 가능)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

#8. CatBoost 모델 정의 및 학습
model = CatBoostClassifier(random_state=42, iterations=1000, learning_rate=0.1, verbose=0)
model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50)

#9. 검증 데이터 평가
y_val_pred = model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
val_f1 = f1_score(y_val, y_val_pred)

print(f"Validation Accuracy: {val_accuracy:.2f}")
print(f"Validation F1 Score: {val_f1:.2f}")

#10. 테스트 데이터 평가
y_test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)

print(f"Test Accuracy: {test_accuracy:.2f}")
print(f"Test F1 Score: {test_f1:.2f}")

#11. 분류 보고서 출력
print("\nClassification Report on Test Data:")
print(classification_report(y_test, y_test_pred))

Label Distribution:
Label
1    19000
0     3000
Name: count, dtype: int64
Validation Accuracy: 0.91
Validation F1 Score: 0.95
Test Accuracy: 0.91
Test F1 Score: 0.95

Classification Report on Test Data:
              precision    recall  f1-score   support

           0       0.82      0.43      0.56       450
           1       0.92      0.99      0.95      2850

    accuracy                           0.91      3300
   macro avg       0.87      0.71      0.76      3300
weighted avg       0.90      0.91      0.90      3300



In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

#2. 데이터 로드
data = pd.read_csv("../../../../monunmon.csv")  # 데이터셋 경로 입력
data['Label'] = data['Label'].apply(lambda x: 1 if x >= 0 else 0)  # 이진 레이블 변환

#3. 특성과 레이블 분리
X = data.drop(columns=['Label'])  # 피처 데이터
y = data['Label']  # 레이블 데이터

#4. Train, Validation, Test 데이터 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

#5. 데이터 정규화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

### 방법 1: 클래스 가중치 적용 ###
print("=== CatBoost with Class Weights ===")
model_weighted = CatBoostClassifier(
    random_state=42, iterations=1000, learning_rate=0.1,
    class_weights=[1/sum(y_train == 0), 1/sum(y_train == 1)],  # 클래스 가중치 적용
    verbose=0
)
model_weighted.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50)

# 검증 데이터 평가
y_val_pred = model_weighted.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
val_f1 = f1_score(y_val, y_val_pred)
print(f"Validation Accuracy: {val_accuracy:.2f}")
print(f"Validation F1 Score: {val_f1:.2f}")

# 테스트 데이터 평가
y_test_pred = model_weighted.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy:.2f}")
print(f"Test F1 Score: {test_f1:.2f}")

# 분류 보고서
y_test_pred = model_weighted.predict(X_test)
print("\nClassification Report on Test Data (Class Weights):")
print(classification_report(y_test, y_test_pred))

### 방법 2: SMOTE를 이용한 오버샘플링 ###
print("\n=== CatBoost with SMOTE ===")
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print("Resampled Training Set Size (SMOTE):", X_train_smote.shape)

model_smote = CatBoostClassifier(random_state=42, iterations=1000, learning_rate=0.1, verbose=0)
model_smote.fit(X_train_smote, y_train_smote, eval_set=(X_val, y_val), early_stopping_rounds=50)

# 검증 데이터 평가
y_val_pred_smote = model_smote.predict(X_val)
val_accuracy_smote = accuracy_score(y_val, y_val_pred_smote)
val_f1_smote = f1_score(y_val, y_val_pred_smote)
print(f"Validation Accuracy: {val_accuracy_smote:.2f}")
print(f"Validation F1 Score: {val_f1_smote:.2f}")

# 테스트 데이터 평가
y_test_pred_smote = model_smote.predict(X_test)
test_accuracy_smote = accuracy_score(y_test, y_test_pred_smote)
test_f1_smote = f1_score(y_test, y_test_pred_smote)
print(f"Test Accuracy: {test_accuracy_smote:.2f}")
print(f"Test F1 Score: {test_f1_smote:.2f}")

# 분류 보고서
y_test_pred_smote = model_smote.predict(X_test)
print("\nClassification Report on Test Data (SMOTE):")
print(classification_report(y_test, y_test_pred_smote))

### 방법 3: 언더샘플링 ###
print("\n=== CatBoost with Undersampling ===")
undersampler = RandomUnderSampler(random_state=42)
X_train_under, y_train_under = undersampler.fit_resample(X_train, y_train)

print("Resampled Training Set Size (Undersampling):", X_train_under.shape)

model_under = CatBoostClassifier(random_state=42, iterations=1000, learning_rate=0.1, verbose=0)
model_under.fit(X_train_under, y_train_under, eval_set=(X_val, y_val), early_stopping_rounds=50)

# 검증 데이터 평가
y_val_pred_under = model_under.predict(X_val)
val_accuracy_under = accuracy_score(y_val, y_val_pred_under)
val_f1_under = f1_score(y_val, y_val_pred_under)
print(f"Validation Accuracy: {val_accuracy_under:.2f}")
print(f"Validation F1 Score: {val_f1_under:.2f}")

# 테스트 데이터 평가
y_test_pred_under = model_under.predict(X_test)
test_accuracy_under = accuracy_score(y_test, y_test_pred_under)
test_f1_under = f1_score(y_test, y_test_pred_under)
print(f"Test Accuracy: {test_accuracy_under:.2f}")
print(f"Test F1 Score: {test_f1_under:.2f}")

# 분류 보고서
y_test_pred_under = model_under.predict(X_test)
print("\nClassification Report on Test Data (Undersampling):")
print(classification_report(y_test, y_test_pred_under))


=== CatBoost with Class Weights ===
Validation Accuracy: 0.88
Validation F1 Score: 0.93
Test Accuracy: 0.87
Test F1 Score: 0.92

Classification Report on Test Data (Class Weights):
              precision    recall  f1-score   support

           0       0.51      0.72      0.60       450
           1       0.95      0.89      0.92      2850

    accuracy                           0.87      3300
   macro avg       0.73      0.80      0.76      3300
weighted avg       0.89      0.87      0.88      3300


=== CatBoost with SMOTE ===
Resampled Training Set Size (SMOTE): (26600, 15)
Validation Accuracy: 0.89
Validation F1 Score: 0.94
Test Accuracy: 0.88
Test F1 Score: 0.93

Classification Report on Test Data (SMOTE):
              precision    recall  f1-score   support

           0       0.55      0.67      0.61       450
           1       0.95      0.91      0.93      2850

    accuracy                           0.88      3300
   macro avg       0.75      0.79      0.77      3300
weigh